In [1]:
# Uses code from 'make j from scratch'
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)

def HsubIstrike(n,m):
    jay = J(n+1,m)
    numerator = x^m*derivative(jay,x)^m
    denominator = jay^(m-1)*(jay-1)
    power=1/(m-2)
    answer0=S((numerator/denominator)^power).truncate(n).polynomial(x)
    al=answer0.list()
    dg=answer0.degree()
    sm=sum(al[k]*(2^6*m^3*x)^k for k in [0..dg])*(-1)^m
    return sm

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def sigmaOdd(n):
    sum=0
    dvs=divisors(n)
    lnth=len(dvs)
    for k in [0..lnth-1]:
        dv=dvs[k]
        if mod(dv,2)==1:
            sum=sum+dv
    return sum

In [2]:
j(10,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4 + 333202640600*x^5 + 4252023300096*x^6 + 44656994071935*x^7 + 401490886656000*x^8 + 3176440229784420*x^9

In [2]:
n=5
for m in [3..6]:
    print HsubIstrike(n,m) # will have to normalize these to obtain "valid" interpolating polynomials.
    print

-532728*x^4 - 122976*x^3 - 16632*x^2 - 504*x + 1

-193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1

-29091725000/3*x^4 - 276380000/3*x^3 - 351000*x^2 - 2600*x + 1

-125627793408*x^4 - 528482304*x^3 - 884736*x^2 - 4608*x + 1



In [4]:
n=5
for m in [3..6]:
    print HsubIstrike(n,m)*(-1)^m
    print

532728*x^4 + 122976*x^3 + 16632*x^2 + 504*x - 1

-193986560*x^4 - 9175040*x^3 - 102400*x^2 - 1280*x + 1

29091725000/3*x^4 + 276380000/3*x^3 + 351000*x^2 + 2600*x - 1

-125627793408*x^4 - 528482304*x^3 - 884736*x^2 - 4608*x + 1



In [ ]:
# print output snipped.
import pickle
import time
wfile=open('/Users/barrybrent/run11aug21no6.txt','w')
data=[]
start=time.time()/60
for m in [3..323]:
    poly=HsubIstrike(101,m)
    data=data+[[m,poly]]
    finish=time.time()/60
    print([m,poly.degree(),finish-start])
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [ ]:
# print output snipped.
rfile=open('/Users/barrybrent/run11aug21no6.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)


wfile = open('/Users/barrybrent/run11jaug21no8.txt','w') #HsubIstrike polynomials
polydata=[]
start=time.time()/60
for n in [0..100]:
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1]
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf]]
    rl=R.lagrange_polynomial(data)
    poly=rl.polynomial(x)
    polydata=polydata+[[n,poly]] 
    finish=time.time()/60
    print(n,poly.degree(),finish-start)
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

In [6]:
import pickle
rfile = open('/Users/barrybrent/run11jaug21no8.txt','r') #HsubIstrike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    dg=poly.degree() # under D coercion, degree.() takes no argument.
    data=data+[3*n-dg]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
f=D(HsubIstrike(10,3))
print f.list()

[1, -504, -16632, -122976, -532728, -1575504, -4058208, -8471232, -17047800, -29883672]


In [4]:
vector([2,2])-vector([1,1])

(1, 1)

In [11]:
f=D(HsubIstrike(10,3))
fl= vector(f.list())
dvs=[1];
for n in [1..9]:
    dvs=dvs+[-504*sigma(n,5)]
dvs=vector(dvs)
print dvs
print dvs-fl

(1, -504, -16632, -122976, -532728, -1575504, -4058208, -8471232, -17047800, -29883672)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [13]:
f=D(HsubIstrike(100,3))
fl= vector(f.list())
dvs=[1];
for n in [1..99]:
    dvs=dvs+[-504*sigma(n,5)]
dvs=vector(dvs)
print dvs-fl

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
